In [1]:
from keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import os
import numpy as np

# Size of the image: 48x48 pixels
pic_size = 48                            #Specifies the size of each image (48x48 pixels in this case)


# Input path for the images
base_path = "/Users/pratiksha/Documents/Pratiksha/GitHub/Face-expression-recognition-with-Deep-Learning/images"

# Function to preprocess images
def preprocess_image(img_path, target_size=(48, 48)):
    """Load and preprocess an image."""
    img = load_img(img_path, target_size=target_size, color_mode="grayscale")  # Grayscale for custom CNNs
    img_array = img_to_array(img)  # Convert to NumPy array
    img_array /= 255.0  # Normalize pixel values
    return img_array

# Function to load and preprocess dataset
def load_dataset(base_path, subset='train'):
    """Load and preprocess the dataset."""
    images = []
    labels = []
    label_map = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}

    subset_path = os.path.join(base_path, subset)
    for expression in os.listdir(subset_path):
        expression_path = os.path.join(subset_path, expression)
        if not os.path.isdir(expression_path):
            continue  # Skip non-directory files

        for img_name in os.listdir(expression_path):
            img_path = os.path.join(expression_path, img_name)
            try:
                img_array = preprocess_image(img_path)
                images.append(img_array)
                labels.append(label_map[expression])  # Use label_map safely
            except Exception as e:
                print(f"Error loading image {img_name}: {e}")

    return np.array(images), np.array(labels)



# Function to visualize sample images
def visualize_samples(base_path, subset='train'):
    """Visualize sample images for each expression."""
    plt.figure(figsize=(12, 20))           # Specifies the size of the plot
    counter = 0                            # Used to count the number of images displayed -- Counter variable
    subset_path = os.path.join(base_path, subset)
    
    for expression in os.listdir(subset_path):                      # Iterates through each expression folder (angry, disgust, etc.) within the train directory.
        expression_path = os.path.join(subset_path, expression)
        if not os.path.isdir(expression_path):
            continue                                                # Skip if it's not a directory
        
        for i, img_name in enumerate(os.listdir(expression_path)[:5]):  # Loops through the first 5 images (i ranges from 1 to 5) within each expression folder.
            counter += 1                                                # Increments the counter variable
            plt.subplot(7, 5, counter)                                  # Creates a subplot grid with 7 rows and 5 columns, placing the current image in position cpt.
            img_path = os.path.join(expression_path, img_name)
            img_array = preprocess_image(img_path)
            plt.imshow(img_array, cmap="gray")                          # Displays the image with a grayscale colormap.
            plt.xlabel(expression)
    
    plt.tight_layout()
    plt.show()


# Function to count images per expression
def count_images_per_expression(base_path, subset='train'):
    """Count and display the number of images per expression."""
    subset_path = os.path.join(base_path, subset)
    for expression in os.listdir(subset_path):
        expression_path = os.path.join(subset_path, expression)
        if not os.path.isdir(expression_path):
            continue
        
        num_images = len(os.listdir(expression_path))
        print(f"{num_images} {expression} images")


In [2]:
count_images_per_expression(base_path, subset='train')
##visualize_samples(base_path, subset='train')

7164 happy images
4938 sad images
4103 fear images
3205 surprise images
4982 neutral images
3993 angry images
436 disgust images


In [3]:
count_images_per_expression(base_path, subset='validation')
#visualize_samples(base_path, subset='validation')

1825 happy images
1139 sad images
1018 fear images
797 surprise images
1216 neutral images
960 angry images
111 disgust images


In [ ]:
#### Error handling:---- don't run now
#for img_name in os.listdir(expression_path):
   # img_path = os.path.join(expression_path, img_name)
   # try:
    #    img_array = preprocess_image(img_path)
     #   images.append(img_array)
     #   labels.append(label_map[expression])
   # except Exception as e:
   #     print(f"Error loading image {img_name}: {e}")


In [5]:
train_images, train_labels = load_dataset(base_path, subset='train')
test_images, test_labels = load_dataset(base_path, subset='validation')

print("Training images shape:", train_images.shape)
print("Training labels shape:", train_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)


Training images shape: (28821, 48, 48, 1)
Training labels shape: (28821,)
Test images shape: (7066, 48, 48, 1)
Test labels shape: (7066,)


-----------------------------------------------------------------------------------

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Size of the image: 48x48 pixels
pic_size = 48                                                       #Specifies the size of each image (48x48 pixels in this case)


# Input path for the images
base_path = "/Users/pratiksha/Documents/Pratiksha/GitHub/Face-expression-recognition-with-Deep-Learning/images"

# number of images to feed into the NN for every batch
batch_size = 16     #Specifies the number of images to feed into the neural network for every batch during training and validation.

train_datagen = ImageDataGenerator(
    rescale=1.0/255,              # Normalize pixel values to [0, 1]
    rotation_range=20,            # Randomly rotate images in the range (degrees)
    width_shift_range=0.2,        # Randomly translate images horizontally
    height_shift_range=0.2,       # Randomly translate images vertically
    shear_range=0.2,              # Apply shearing transformations
    zoom_range=0.2,               # Randomly zoom images
    horizontal_flip=True,         # Randomly flip images horizontally
    fill_mode='nearest'           # Filling method for points outside boundaries
)

                            #ImageDataGenerator : it is used to generate a batch of images with some random transformations
validation_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:

train_generator = train_datagen.flow_from_directory(base_path + "/train",            # flow_from_directory: Generates batches of augmented/normalized data from image files in the train directory.
                                                    target_size=(pic_size,pic_size), # Resizes images to (pic_size, pic_size) pixels.
                                                    color_mode="grayscale",           # Converts images to grayscale format.
                                                    batch_size=batch_size,            # Number of images per batch to be yielded from the generator.
                                                    class_mode='categorical',         # Returns one-hot encoded labels for multi-class classification.
                                                    shuffle=True)                     #  Shuffles the order of images after every epoch.


In [ ]:
validation_generator = validation_datagen.flow_from_directory(base_path + "/validation", # fetches batches of validation data from the validation directory.
                                                    target_size=(pic_size,pic_size),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=False)

In [ ]:
# Compute class weights for the training data
classes = train_generator.classes  # Extract class indices from the train generator
class_weights = compute_class_weight('balanced', classes=np.unique(classes), y=classes)
class_weights_dict = dict(enumerate(class_weights))
print("Computed class weights:", class_weights_dict)

-----------------------------------------------------------------------------------

In [ ]:
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam

# Number of labels :
num_labels =  7

 # Creating a sequential model :

model = Sequential()

In [ ]:
# 1st Convolution layer
model.add(Conv2D(36,(3,3), padding='same', input_shape=(48, 48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))  #read max pooling adv gpt****************************************
model.add(Dropout(0.25)) #add the range table for dropout
model.summary()

In [ ]:
# 2nd Convolution layer
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 3rd Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 4th Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [ ]:
# Fully connected layer 1st layer
model.add(Flatten())
model.add(Dense(128, activation='relu')) # This is sufficient to prepare the output for dense layers.
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # 7 output units for 7 classes.

In [ ]:
opt = Adam(learning_rate=0.0001)                                                                 #optimizer: Specifies the optimizer (Adam in this case) to use for training.
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])   #metrics: Specifies the metrics to evaluate the model (accuracy in this case).
     #loss: Specifies the loss function (categorical crossentropy for multi-class classification).
     #categorical_crossentropy : Computes the cross-entropy loss between true labels and predicted labels.
     #crossentropy : measures the performance of a classification model whose output is a probability value between 0 and 1.

In [ ]:
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
epochs = 20
# Specifies the number of epochs to train the model. Each epoch means one complete pass through the entire training dataset.

from keras.callbacks import ModelCheckpoint
# Imports the ModelCheckpoint class from keras.callbacks. This is used to save the model at certain points during training.

# Define the checkpoint to save the best model based on validation accuracy
checkpoint = ModelCheckpoint(
    "model_weights.keras",  # The file path where the model weights will be saved. Updated to end with .keras for clarity.
    monitor='val_accuracy',  # Metric to be monitored. 'val_accuracy' means validation accuracy.
    verbose=1,  # Verbosity mode. 1 means that messages will be printed when the model is being saved.
    save_best_only=True,  # If True, the latest best model according to the monitored metric will not be overwritten.
    mode='max'  # Mode for the monitored metric. 'max' means that the model will be saved when the quantity monitored has stopped increasing.
)
callbacks_list = [checkpoint]
# A list of callbacks to pass to the model during training. Here, it contains only the checkpoint callback.

# Train the model using model.fit instead of model.fit_generator
history = model.fit(
    x=train_generator,  # The training data. Here, train_generator is used to provide batches of data during training.
    epochs=epochs,  # The number of epochs to train the model.
    steps_per_epoch=train_generator.n // train_generator.batch_size,  
    # The number of steps per epoch. This is the total number of samples in the training data divided by the batch size.
    
    validation_data=validation_generator,  # The validation data. Here, validation_generator is used to provide batches of validation data.
    validation_steps=validation_generator.n // validation_generator.batch_size,  
    # The number of validation steps per epoch. This is the total number of samples in the validation data divided by the batch size.
    
    callbacks=callbacks_list  # List of callbacks to apply during training. Here, it includes the checkpoint callback to save the best model.
)

# Explanation for validation_generator.n // validation_generator.batch_size:
# This calculates the number of batches (steps) needed to go through the entire validation dataset once.
# - validation_generator.n is the total number of samples in the validation dataset.
# - validation_generator.batch_size is the number of samples in each batch.
# The integer division (//) ensures we get the whole number of batches. For example, if there are 1000 validation samples and the batch size is 32,
# this would be 1000 // 32 = 31 steps per epoch for validation.


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))

# Plot Loss
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer: Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')

plt.show()


In [ ]:
# Show the confusion matrix of our predictions

# Compute predictions
predictions = model.predict(validation_generator)
y_pred = [np.argmax(probas) for probas in predictions]
y_test = validation_generator.classes
class_names = list(validation_generator.class_indices.keys())

from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Normalized confusion matrix')
plt.show()


In [ ]:

model.save('/Users/pratiksha/Documents/Pratiksha/Documents/GitHub/GitHub/Face-expression-recognition-with-Deep-Learning/my_model.keras')
